# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-06 06:05:26] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=39987, mem_fraction_static=0.836, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=412665427, constrained_json_whitespace_pattern=None, wa

[2025-06-06 06:05:26] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-06-06 06:05:37] Attention backend not set. Use flashinfer backend by default.
[2025-06-06 06:05:37] Init torch distributed begin.


[2025-06-06 06:05:37] Init torch distributed ends. mem usage=0.00 GB


[2025-06-06 06:05:38] Load weight begin. avail mem=77.90 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.27s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.27s/it]

[2025-06-06 06:05:45] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.82 GB, mem usage=0.93 GB.
[2025-06-06 06:05:45] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-06-06 06:05:45] Memory pool end. avail mem=37.50 GB


[2025-06-06 06:05:45] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=37.73 GB
[2025-06-06 06:05:45] INFO:     Started server process [2328503]
[2025-06-06 06:05:45] INFO:     Waiting for application startup.
[2025-06-06 06:05:45] INFO:     Application startup complete.
[2025-06-06 06:05:45] INFO:     Uvicorn running on http://127.0.0.1:39987 (Press CTRL+C to quit)


[2025-06-06 06:05:46] INFO:     127.0.0.1:36750 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-06 06:05:46] INFO:     127.0.0.1:36764 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-06 06:05:46] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-06-06 06:05:47,258 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

2025-06-06 06:06:32,749 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-06-06 06:06:32,756 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-06-06 06:06:46,499 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-06-06 06:06:46] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0
2025-06-06 06:06:46,930 - INFO - flashinfer.jit: Loading JIT ops: cascade
2025-06-06 06:06:46,943 - INFO - flashinfer.jit: Finished loading JIT ops: cascade
2025-06-06 06:06:46,968 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


2025-06-06 06:07:00,453 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


2025-06-06 06:07:02,619 - INFO - flashinfer.jit: Loading JIT ops: quantization


2025-06-06 06:07:16,543 - INFO - flashinfer.jit: Finished loading JIT ops: quantization
[2025-06-06 06:07:16] INFO:     127.0.0.1:36778 - "POST /generate HTTP/1.1" 200 OK
[2025-06-06 06:07:16] The server is fired up and ready to roll!


[2025-06-06 06:07:19] INFO:     127.0.0.1:57090 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-06-06 06:07:19] Child process unexpectedly failed with exitcode=9. pid=2329022


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-06 06:07:27] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=34628, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=804603219, constrained_json_whitespace_pattern=None, watc

[2025-06-06 06:07:27] Infer the chat template name from the model path and obtain the result: llama-2.


[2025-06-06 06:07:34] Attention backend not set. Use flashinfer backend by default.
[2025-06-06 06:07:34] Init torch distributed begin.


[2025-06-06 06:07:34] Init torch distributed ends. mem usage=0.00 GB


[2025-06-06 06:07:35] Load weight begin. avail mem=61.76 GB


...

[2025-06-06 06:07:41] INFO:     127.0.0.1:50872 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-06 06:07:42] INFO:     127.0.0.1:50876 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-06 06:07:42] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-06-06 06:07:42,825 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-06-06 06:07:42,839 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-06-06 06:07:42,846 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-06-06 06:07:42,856 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-06-06 06:07:43,359 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
2025-06-06 06:07:43,371 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


2025-06-06 06:07:45,784 - INFO - flashinfer.jit: Loading JIT ops: quantization
2025-06-06 06:07:45,798 - INFO - flashinfer.jit: Finished loading JIT ops: quantization
[2025-06-06 06:07:45] INFO:     127.0.0.1:50888 - "POST /generate HTTP/1.1" 200 OK
[2025-06-06 06:07:45] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-06 06:07:47] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
2025-06-06 06:07:47,041 - INFO - flashinfer.jit: Loading JIT ops: cascade
2025-06-06 06:07:47,054 - INFO - flashinfer.jit: Finished loading JIT ops: cascade


[2025-06-06 06:07:47] INFO:     127.0.0.1:50890 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-06-06 06:07:47] Child process unexpectedly failed with exitcode=9. pid=2336372


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-06 06:07:54] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=37327, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=142349003, constrained_json_whitespa

[2025-06-06 06:07:54] Casting torch.bfloat16 to torch.float16.


[2025-06-06 06:08:03] Casting torch.bfloat16 to torch.float16.


[2025-06-06 06:08:04] Casting torch.bfloat16 to torch.float16.
[2025-06-06 06:08:04] Attention backend not set. Use flashinfer backend by default.
[2025-06-06 06:08:04] Init torch distributed begin.


[2025-06-06 06:08:05] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-06 06:08:24] INFO:     Started server process [2338078]
[2025-06-06 06:08:24] INFO:     Waiting for application startup.
[2025-06-06 06:08:24] INFO:     Application startup complete.
[2025-06-06 06:08:24] INFO:     Uvicorn running on http://127.0.0.1:37327 (Press CTRL+C to quit)


[2025-06-06 06:08:25] INFO:     127.0.0.1:49224 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-06 06:08:25] INFO:     127.0.0.1:49236 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-06 06:08:25] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-06-06 06:08:26,397 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-06-06 06:08:26,411 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-06-06 06:08:26,418 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-06-06 06:08:26,429 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-06-06 06:08:26,809 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
2025-06-06 06:08:26,821 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


2025-06-06 06:08:28,879 - INFO - flashinfer.jit: Loading JIT ops: quantization
2025-06-06 06:08:28,892 - INFO - flashinfer.jit: Finished loading JIT ops: quantization
[2025-06-06 06:08:28] INFO:     127.0.0.1:49238 - "POST /generate HTTP/1.1" 200 OK
[2025-06-06 06:08:28] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-06 06:08:30] Detected chat template content format: string
[2025-06-06 06:08:30] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
2025-06-06 06:08:30,973 - INFO - flashinfer.jit: Loading JIT ops: cascade
2025-06-06 06:08:30,986 - INFO - flashinfer.jit: Finished loading JIT ops: cascade


[2025-06-06 06:08:31] INFO:     127.0.0.1:39352 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-06-06 06:08:31] Child process unexpectedly failed with exitcode=9. pid=2338636


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-06 06:08:38] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='127.0.0.1', port=32210, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1004714905, constrained_json_whitespace_patte

[2025-06-06 06:08:45] Casting torch.bfloat16 to torch.float16.


[2025-06-06 06:08:45] Casting torch.bfloat16 to torch.float16.
[2025-06-06 06:08:45] Attention backend not set. Use flashinfer backend by default.
[2025-06-06 06:08:45] Init torch distributed begin.


[2025-06-06 06:08:46] Init torch distributed ends. mem usage=0.00 GB


[2025-06-06 06:08:47] Load weight begin. avail mem=78.50 GB


...

[2025-06-06 06:09:06] INFO:     Started server process [2341078]
[2025-06-06 06:09:06] INFO:     Waiting for application startup.
[2025-06-06 06:09:06] INFO:     Application startup complete.
[2025-06-06 06:09:06] INFO:     Uvicorn running on http://127.0.0.1:32210 (Press CTRL+C to quit)
[2025-06-06 06:09:06] INFO:     127.0.0.1:50482 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-06 06:09:07] INFO:     127.0.0.1:50498 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-06 06:09:07] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-06-06 06:09:07,678 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-06-06 06:09:07,693 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-06-06 06:09:07,699 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-06-06 06:09:07,710 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-06-06 06:09:08,081 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
2025-06-06 06:09:08,093 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_f16_dtype_kv_f16_dtype_o_f16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


2025-06-06 06:09:10,101 - INFO - flashinfer.jit: Loading JIT ops: quantization
2025-06-06 06:09:10,114 - INFO - flashinfer.jit: Finished loading JIT ops: quantization
[2025-06-06 06:09:10] INFO:     127.0.0.1:50512 - "POST /generate HTTP/1.1" 200 OK
[2025-06-06 06:09:10] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-06-06 06:09:11] Detected chat template content format: string
[2025-06-06 06:09:11] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
2025-06-06 06:09:11,454 - INFO - flashinfer.jit: Loading JIT ops: cascade
2025-06-06 06:09:11,467 - INFO - flashinfer.jit: Finished loading JIT ops: cascade


[2025-06-06 06:09:11] INFO:     127.0.0.1:38948 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-06-06 06:09:11] Child process unexpectedly failed with exitcode=9. pid=2341310
[2025-06-06 06:09:11] Child process unexpectedly failed with exitcode=9. pid=2341244


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

Overlap scheduler is disabled because of using eagle speculative decoding.
[2025-06-06 06:09:19] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=37248, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=508590661, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_time

[2025-06-06 06:09:27] Attention backend not set. Use flashinfer backend by default.
[2025-06-06 06:09:27] Init torch distributed begin.


[2025-06-06 06:09:29] Init torch distributed ends. mem usage=0.00 GB


[2025-06-06 06:09:30] Load weight begin. avail mem=78.50 GB


[2025-06-06 06:09:33] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-06-06 06:09:39] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-06-06 06:09:39] INFO:     Started server process [2342128]
[2025-06-06 06:09:39] INFO:     Waiting for application startup.
[2025-06-06 06:09:39] INFO:     Application startup complete.
[2025-06-06 06:09:39] INFO:     Uvicorn running on http://0.0.0.0:37248 (Press CTRL+C to quit)
[2025-06-06 06:09:39] INFO:     127.0.0.1:39694 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-06 06:09:40] INFO:     127.0.0.1:52212 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-06 06:09:40] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-06-06 06:09:41,280 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-06-06 06:09:41,294 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-06-06 06:09:41,301 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-06-06 06:09:41,311 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-06-06 06:09:42,297 - INFO - flashinfer.jit: Loading JIT ops: quantization
2025-06-06 06:09:42,309 - INFO - flashinfer.jit: Finished loading JIT ops: quantization
[2025-06-06 06:09:42] INFO:     127.0.0.1:52224 - "POST /generate HTTP/1.1" 200 OK
[2025-06-06 06:09:42] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-06 06:09:44] Detected chat template content format: string
[2025-06-06 06:09:44] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-06 06:09:45] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, accept len: 1.76, cuda graph: False, gen throughput (token/s): 11.48, #queue-req: 0


[2025-06-06 06:09:46] Decode batch. #running-req: 1, #token: 163, token usage: 0.01, accept len: 1.80, cuda graph: False, gen throughput (token/s): 109.82, #queue-req: 0


[2025-06-06 06:09:46] Decode batch. #running-req: 1, #token: 229, token usage: 0.01, accept len: 1.65, cuda graph: False, gen throughput (token/s): 98.56, #queue-req: 0


[2025-06-06 06:09:47] Decode batch. #running-req: 1, #token: 297, token usage: 0.01, accept len: 1.70, cuda graph: False, gen throughput (token/s): 101.16, #queue-req: 0


[2025-06-06 06:09:48] Decode batch. #running-req: 1, #token: 360, token usage: 0.02, accept len: 1.57, cuda graph: False, gen throughput (token/s): 94.89, #queue-req: 0


[2025-06-06 06:09:48] Decode batch. #running-req: 1, #token: 426, token usage: 0.02, accept len: 1.65, cuda graph: False, gen throughput (token/s): 100.14, #queue-req: 0


[2025-06-06 06:09:49] INFO:     127.0.0.1:52238 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-06-06 06:09:49] Child process unexpectedly failed with exitcode=9. pid=2342340


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).